In [1]:
import numpy as np
import pandas as pd
import pickle
# SQL related packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
# sklearn packages
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics.scorer import make_scorer
# text analysis packages
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim.models import word2vec, Word2Vec

### Load non-text features

In [2]:
# read the data with the non-text features
patents = pd.read_pickle("patent_data/nontext_features.p")

In [3]:
print patents.shape
patents.head()

(12030, 16)


,id,publication_year,B,C,D,E,F,G,H,num_applications,num_patent_citations,num_nonpatent_citations,num_claims,num_similar_doc,num_authors,payment_times
0,US6699658B1,2004,0,1,0,0,0,0,0,5,28,34,42,1,4,3
1,US6699724B1,2004,0,0,0,0,0,1,0,32,47,44,25,0,4,3
2,US6690816B2,2004,0,0,0,0,0,1,0,8,9,0,32,1,4,1
3,US6711436B1,2004,0,0,0,0,0,0,0,4,105,109,45,7,1,3
4,US6711432B1,2004,0,0,0,0,0,0,0,7,15,114,44,3,4,3


#### Format the data

In [4]:
# reformat the response variable into binary
y_data = np.zeros(patents.shape[0])
y_data[patents['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.62859517872


In [5]:
# predictors
x_data = patents.drop(['id', 'payment_times', 'publication_year'], axis = 1).values
x_data.shape

(12030, 13)

### Load claims data

In [6]:
# access to sql database
dbname = 'patent_db'
username = 'jy'
pswd = 'jy'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# reading from sql database
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [7]:
# read data from 2004-2007
years = np.arange(2004, 2008)

# dataframe to store the results
claims = pd.DataFrame()

# import the abstract from each table
for year in years:
    # query:
    sql_query = """
    SELECT claims, id, payment_times, classification
        FROM patents_%s;
    """ %str(year)

    results = pd.read_sql_query(sql_query,con)
    
    claims = pd.concat([claims, results], axis = 0)
    
# check size of the data
claims.shape

(12033, 4)

In [8]:
# remove the patents missing classification data
missing_class_index = (claims['classification'].values == 'NA')

# reassign patent index
claims.index = range(len(claims.index))
# drop the rows
claims =  claims.drop(patents.index[missing_class_index])
claims.shape

(12030, 4)

In [9]:
def tokenize_cleaning(text):
    # tokenize the text first
    tokens = word_tokenize(text.decode('utf-8'))
    
    # lowercase all the words
    tokens = [w.lower() for w in tokens]
    
    # clean up stop words and punctuations 
    stop_list = stopwords.words('english') + list(string.punctuation)

    tokens_no_stop = [token for token in tokens
                        if token not in stop_list]            
    
#     # extract stem of the words
#     stemmer = PorterStemmer()
#     tokens_stem = [stemmer.stem(token) for token in tokens_no_stop]

    # use lemma instead
    # reason: remove the influence of plural or tense
    # but retain the subtle difference in legal writting
    lemmatizer = WordNetLemmatizer()
    tokens_lemma = [lemmatizer.lemmatize(token) for token in tokens_no_stop]
    
    # remove numbers (the actual values are not useful)
    tokens_no_num = []
    for token in tokens_lemma:
        try:
            float(token)
        except:
            tokens_no_num.append(token)
    
    return tokens_no_num

In [10]:
# tokenize_clean the claims and count the occurence of the words

cleaned_text = []
for i in range(claims.shape[0]):
    tokens = tokenize_cleaning(claims['claims'].iloc[i])
    cleaned_text.append(tokens)

In [11]:
word2vec_model = Word2Vec.load('models/word2vec_claims_final')

In [12]:
# compute average word-vector for a text
def dec_vec(model, text):
    # store the vector for each word
    vectors = []
    
    # compute on each word
    for j in range(len(text)):
        try:
            vectors.append(model.wv[text[j]])
        except:
            continue
    
    if not vectors:
        vectors_mean = np.zeros((1, 100))
    else:
        vectors_mean = np.nanmean(vectors, axis = 0)
        vectors_mean = vectors_mean.reshape((1, 100))
    
    # return vector mean
    return vectors_mean

In [13]:
# compute a vector for each patent
claims_vec = np.zeros((1, 100)) #initialize

for i in range(claims.shape[0]):
    vec = dec_vec(word2vec_model, cleaned_text[i])
    claims_vec = np.concatenate([claims_vec, vec], axis = 0)

# remove the first line
claims_vec = claims_vec[1:, :]
claims_vec.shape

(12030, 100)

### Combine text and non-text features

In [14]:
# combine the claims data with the non-text features
x_data = np.concatenate([x_data, claims_vec], axis = 1)

In [15]:
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size = 2000, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (10030, 113)
x_test:  (2000, 113)
y_train:  (10030,)
y_test:  (2000,)


In [16]:
### subsampling the training data
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_train == 0)

#random shuffle the rows
n = x_train.shape[0]
perm = range(n)
np.random.shuffle(perm)

x_train = x_train[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = x_train[y_train == 1, :]
x_not_useful = x_train[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size]
y_useful = y_useful[:num_size]

# combine the two classes
x_train_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_train_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_train_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_train_sub = x_train_sub[perm2]
y_train_sub = y_train_sub[perm2]

# check the size
print x_train_sub.shape
print y_train_sub.shape

(7506, 113)
(7506,)


In [17]:
# # standardize the predictors
# scaler = preprocessing.StandardScaler()

# x_train_std = scaler.fit_transform(x_train_sub)
# x_test_std = scaler.transform(x_test)

In [18]:
# load the saved scaler
scaler = pickle.load(open('models/final_model_scaler.p', 'r'))
x_test_std = scaler.transform(x_test)

### Model training

In [19]:
# write my own scoring function
def my_loss_func(y_actual, y_pred):
    
    # set the price of applying for each patent
    patent_cost = 20
    
    # set the value of a useful patent
    useful_value = 30
    
    total_cost = np.sum(y_pred == 1) * patent_cost
    total_value = np.sum((y_actual == 1) & (y_pred == 1)) * useful_value
    
    profit = total_value -  total_cost
    
    return profit

my_scorer = make_scorer(my_loss_func, greater_is_better=True)

#### Random Forest

In [ ]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 10, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_std, y_train_sub)

In [20]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: -1113.03756994
Best parameter:  {'max_features': 6}


In [20]:
# load saved final model
best_rf = pickle.load(open('models/final_model.p', 'r'))

In [21]:
# # check confusion matrix
# best_rf = grid_model.best_estimator_
# best_rf.fit(x_train_std, y_train_sub)

y_pred = best_rf.predict(x_test_std)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.569
F1 score:  0.622922134733
Precision:  0.554085603113
Recall:  0.711288711289
Net value:  1340


array([[426, 289],
       [573, 712]])

In [25]:
# tune class weight of random forest
model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 8, 2)

# add class weight tuning to the random forest model
weights = np.logspace(-3,3,7)
weight_list_dict = [{0:1, 1: weights[i]} for i in range(len(weights))]

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space,
                                       'class_weight': weight_list_dict}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_std, y_train_sub)

In [26]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: -913.073541167
Best parameter:  {'max_features': 2, 'class_weight': {0: 1, 1: 1000.0}}


In [28]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_std, y_train_sub)

y_pred = best_rf.predict(x_test_std)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.5095
F1 score:  0.526772793054
Precision:  0.692893401015
Recall:  0.424902723735
Net value:  310


array([[473, 242],
       [739, 546]])

In [35]:
# check where the model can get if selected by accuracy

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 10, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train_std, y_train_sub)

In [36]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.569411137756
Best parameter:  {'max_features': 8}


In [37]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_std, y_train_sub)

y_pred = best_rf.predict(x_test_std)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.574
F1 score:  0.628919860627
Precision:  0.56186770428
Recall:  0.714144411474
Net value:  720


array([[426, 289],
       [563, 722]])

### Some benchmark

In [22]:
# if everything is predicted to be useful (benchmark)
y_pred_uniform = np.ones(y_pred.shape)
# accuracy
print "Test accuracy: ", np.mean(y_pred_uniform == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred_uniform)
print "Precision: ", metrics.precision_score(y_test, y_pred_uniform)
print "Recall: ", metrics.recall_score(y_test, y_pred_uniform)
print "Net value: ", my_loss_func(y_test, y_pred_uniform)
metrics.confusion_matrix(y_test, y_pred_uniform)

Test accuracy:  0.6425
F1 score:  0.782343987823
Precision:  0.6425
Recall:  1.0
Net value:  -1450


array([[   0,  715],
       [   0, 1285]])

In [24]:
# if randomly assign the two classes with the same ratio
y_pred_random = np.random.uniform(0, 1, size = y_test.shape[0])
# turn into binary
y_random_class = np.zeros(y_test.shape[0]) 
y_random_class[y_pred_random > 1- np.mean(y_test)] = 1

print "Test accuracy: ", np.mean(y_random_class == y_test)
print "F1 score: ", metrics.f1_score(y_random_class, y_test)
print "Precision: ", metrics.precision_score(y_test, y_random_class)
print "Recall: ", metrics.recall_score(y_test, y_random_class)
print "Net value: ", my_loss_func(y_test, y_random_class)
metrics.confusion_matrix(y_test, y_random_class)

Test accuracy:  0.539
F1 score:  0.64068589244
Precision:  0.64168618267
Recall:  0.639688715953
Net value:  -960


array([[256, 459],
       [463, 822]])